In [61]:
# get all the data from the site https://www.epfl.ch/campus/associations/list/

import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


url = 'https://www.epfl.ch/campus/associations/list/'

response = requests.get(url, headers=headers)
print(f'Status code: {response.status_code}')
print(f'Content type: {response.headers["content-type"]}')
#print(f'text: {response.text}')
soup = BeautifulSoup(response.text, 'html.parser')

# get the table

tables = soup.find_all('ul')
wanted_table = tables[2]


# get the rows
items = wanted_table.find_all('li')

# get the data
#when possible for each we want url and name and fullname
#most look like this -> <li><strong><a href="http://adec.epfl.ch/" rel="noopener noreferrer" target="_blank">ADEC</a></strong> (Association of EPFL Students in Chemistry)</li>
# some don't have a link -> <li><strong>AGEPoly</strong> (General Association of EPFL Students)</li>
#and some are wrapped in a span -> <li><span><strong><a href="http://www.aiesec.ch/epfl" rel="noopener noreferrer" target="_blank">AIESEC</a></strong> (International Association of Students in Economic and Commercial Sciences)</span></li>
data = {}
for item in items:
    # Find the first <a> tag
    link = item.find('a')
    text = item.text.strip()[:-1]
    parts = text.split('(')
    acronym = parts[0].strip()
    try:
        fullname = parts[1].strip()
    except IndexError:
        fullname = acronym
    # Extract acronym, URL, and full name if the <a> tag exists
    if link:
        url = link.get('href', 'N/A')
    else:
        # If <a> tag does not exist, extract text and assume it contains both acronym and full name
        url = 'N/A'

    data[acronym] = {'url': url, 'fullname': fullname}
    # Extract full name

# write to csv
with open('epfl_associations.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['acronym', 'url', 'fullname'])
    for acronym, info in data.items():
        writer.writerow([acronym, info['url'], info['fullname']])







Status code: 200
Content type: text/html; charset=UTF-8
